# Benchmark models

In [7]:

# Imports
## Basics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook
import os
from functools import partial

## Views 3
import views_runs
from viewser.operations import fetch
from views_forecasts.extensions import *
from viewser import Queryset, Column



In [8]:

# Common parameters:

dev_id = 'Fatalities002'
run_id = 'Fatalities002'
EndOfHistory = 508
get_future = False

username = os.getlogin()

steps = [*range(1, 36+1, 1)] # Which steps to train and predict for

fi_steps = [1,3,6,12,36]
# Specifying partitions

calib_partitioner_dict = {"train":(121,396),"predict":(397,456)}
test_partitioner_dict = {"train":(121,444),"predict":(457,504)}
future_partitioner_dict = {"train":(121,492),"predict":(505,512)}
calib_partitioner =  views_runs.DataPartitioner({"calib":calib_partitioner_dict})
test_partitioner =  views_runs.DataPartitioner({"test":test_partitioner_dict})
future_partitioner =  views_runs.DataPartitioner({"future":future_partitioner_dict})

Mydropbox = f'/Users/{username}/Dropbox (ViEWS)/ViEWS/'
overleafpath = f'/Users/{username}/Dropbox (ViEWS)/Apps/Overleaf/Prediction competition 2023/'


print('Dropbox path set to',Mydropbox)
print('Overleaf path set to',overleafpath)

# Benchmark model parameters
filepath = Mydropbox + 'Prediction_competition_2023/'

year_list = [2018, 2019, 2020, 2021]
draws_cm = 1000
draws_pgm = 100

steps = [3,4,5,6,7,8,9,10,11,12,13,14]
stepcols = ['ln_ged_sb_dep']
for step in steps:
    stepcols.append('step_pred_' + str(step))
print(stepcols)



Dropbox path set to /Users/jim/Dropbox (ViEWS)/ViEWS/
Overleaf path set to /Users/jim/Dropbox (ViEWS)/Apps/Overleaf/Prediction competition 2023/
['ln_ged_sb_dep', 'step_pred_3', 'step_pred_4', 'step_pred_5', 'step_pred_6', 'step_pred_7', 'step_pred_8', 'step_pred_9', 'step_pred_10', 'step_pred_11', 'step_pred_12', 'step_pred_13', 'step_pred_14']


In [9]:
from BenchmarkModels import poisson_expand_single_point_predictions, describe_expanded
from BenchmarkModels import poisson_expand_multiple_point_predictions, save_models, save_actuals
from BenchmarkModels import bootstrap_expand_single_point_predictions

# cm models
## Ensemble model predictions

In [10]:
# Expand ensemble predictions:
ensemble_df = pd.DataFrame.forecasts.read_store('cm_ensemble_genetic_test', run=dev_id)[stepcols]
ensemble_df.head()


sc_predictions_ensemble =  poisson_expand_single_point_predictions(ensemble_df=ensemble_df,level='cm',year_list=year_list,draws=1000)

describe_expanded(df=sc_predictions_ensemble[0]['prediction_df'], df_expanded=sc_predictions_ensemble[0]['expanded_df'], month=457, country=57)   
sc_predictions_ensemble[0]['expanded_df'].head()

pr_46_cm_ensemble_genetic_test.parquet


/Users/jim/Work/ViEWS/ViEWS3/prediction_competition_2023/BenchmarkModels.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['prediction'] = np.expm1(df_temp[this_col].values)
/Users/jim/Work/ViEWS/ViEWS3/prediction_competition_2023/BenchmarkModels.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['prediction'].loc[this_month] = df_temp['prediction'].values
/Users/jim/Work/ViEWS/ViEWS3/prediction_competition_2023/BenchmarkModels.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

2018
2019
2020
2021
Mean and std of original predictions, all rows:
        prediction
count  2292.000000
mean     46.288810
std     334.649615
min       0.000000
25%       0.017667
50%       0.037432
75%       0.190760
max    6384.809995
Mean and std of expanded predictions, all rows:
        outcome
count   2292000
unique     4192
top           0
freq    1764057
Mean and std of original predictions, one cm:
count     2.00000
unique    2.00000
top       1.05692
freq      1.00000
Name: (457, 57), dtype: float64
Mean and std of expanded predictions, one cm:
        outcome
count      1000
unique        7
top           1
freq        376
Variance: outcome    1.056657
dtype: float64


outcome
month_id country_id draw        
457      1          0          0
                    1          0
                    2          0
                    3          0
                    4          0

# Bootstrapped actuals

In [16]:
actuals_by_year =  bootstrap_expand_single_point_predictions(ensemble_df=ensemble_df,draw_from_column='ln_ged_sb_dep',level='cm',year_list=year_list,draws=1000)

describe_expanded(df=actuals_by_year[0]['actuals_df'], df_expanded=actuals_by_year[0]['expanded_df'], month=457, country=57)   

actuals_by_year[0]['expanded_df'].head()

2018
2019
2020
2021
Mean and std of original predictions, all rows:
       ln_ged_sb_dep
count    2292.000000
mean       24.078098
std       188.941513
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max      2964.000000
Mean and std of expanded predictions, all rows:
          outcome
count   2292000.0
unique      136.0
top           0.0
freq    1937410.0
Mean and std of original predictions, one cm:
count     2.0
unique    2.0
top       0.0
freq      1.0
Name: (457, 57), dtype: float64
Mean and std of expanded predictions, one cm:
        outcome
count    1000.0
unique     72.0
top         0.0
freq      828.0
Variance: outcome    43652.126445
dtype: float64


outcome
month_id country_id draw        
457      1          0        0.0
                    1        0.0
                    2        0.0
                    3        0.0
                    4     2368.0

Check that mean and sigma are acceptably close between actuals and bootstraps:

In [17]:
actuals_by_year[0]['actuals_df']['ln_ged_sb_dep'].describe()

count    2292.000000
mean       24.078098
std       188.941513
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max      2964.000000
Name: ln_ged_sb_dep, dtype: float64

In [18]:
pd.DataFrame(actuals_by_year[0]['expanded_df']['outcome']).astype('float64').describe()

,outcome
count,2.292000e+06
mean,2.410475e+01
std,1.887341e+02
min,0.000000e+00
25%,0.000000e+00
50%,0.000000e+00
75%,0.000000e+00
max,2.964000e+03


# Last historical values

In [11]:
qs = (Queryset("benchmark_cm", "country_month")

   # target variable
   .with_column(Column("ged_sb", from_table="ged2_cm", from_column="ged_sb_best_sum_nokgi")
                .transform.missing.fill()
                .transform.missing.replace_na()
                )


   .with_theme("benchmark")
   .describe("""Data for empirical benchmark model, cm level

            """)
   )
df_cm_historical_values = qs.publish().fetch()
df_cm_historical_values['prediction'] = np.nan

for year in year_list:
    for month in range(1,12+1):
        this_month = ((year-1980)*12)+month
        last_month_with_data = ((year-1980)*12)-2
        this_shift = this_month - last_month_with_data
#        print(year,month,last_month_with_data, this_shift)
        
        df_cm_historical_values['prediction'].loc[this_month] = df_cm_historical_values.groupby(['country_id'])['ged_sb'].shift(this_shift)

df_cm_historical_values = df_cm_historical_values.loc[454:504]

 .    

In [12]:

sc_predictions_last_historical =  poisson_expand_single_point_predictions(ensemble_df=df_cm_historical_values,level='cm',year_list=year_list,draws=1000)

describe_expanded(df=sc_predictions_last_historical[0]['prediction_df'], df_expanded=sc_predictions_last_historical[0]['expanded_df'], month=457, country=57)   
sc_predictions_last_historical[0]['expanded_df'].head()

KeyError: 'step_pred_3'

## Constituent model predictions

In [ ]:
# Retrieve the model predictions
# Using code from the VIEWS fatalities002 system

# 20 models: 1 "draw" from each of 20 constituent models, expanded with 50 draws from Poisson distribution for each model.

from ModelDefinitions import DefineEnsembleModels

level = 'cm'
ModelList_cm = DefineEnsembleModels(level)
ModelList_cm = ModelList_cm[0:20] # Drop Markov models
#ModelList_cm = ModelList_cm[0:3] # For debugging

i = 0
for model in ModelList_cm:
    print(i, model['modelname'], model['data_train'])
    i = i + 1

# Retrieving the predictions for calibration and test partitions
# The ModelList contains the predictions organized by model
from Ensembling import CalibratePredictions, RetrieveStoredPredictions, mean_sd_calibrated, gam_calibrated

ModelList_cm = RetrieveStoredPredictions(ModelList_cm, steps, EndOfHistory, dev_id, level, get_future)

ModelList_cm = CalibratePredictions(ModelList_cm, EndOfHistory, steps)

In [ ]:
sc_predictions_constituent =  poisson_expand_multiple_point_predictions(ModelList=ModelList_cm,level='cm',year_list=year_list,draws=1000)


In [ ]:
sc_predictions_constituent[0]['expanded_df'].tail()

# Saving the cm benchmark models

In [ ]:
model_names = ['ensemble','constituent']
model_list = [sc_predictions_ensemble,sc_predictions_constituent]

save_models('cm',model_names,model_list, filepath)
save_actuals('cm',ensemble_df, filepath, year_list)

# pgm level

In [ ]:
# Expand ensemble predictions:
ensemble_df = pd.DataFrame.forecasts.read_store('pgm_ensemble_cm_calib_test', run=dev_id)[stepcols]
ensemble_df.head()


sc_predictions_ensemble_pgm = poisson_expand_single_point_predictions(ensemble_df=ensemble_df,year_list=year_list,draws=100,level='pgm')

sc_predictions_ensemble_pgm[0]['expanded_df'].head()



In [ ]:
# Saving 
model_names = ['ensemble','constituent']
model_list = [sc_predictions_ensemble_pgm]

save_models('pgm',model_names,model_list, filepath)
#save_cm_actuals(ensemble_df, filepath, year_list)


